In [2]:
import requests
import json
from bs4 import BeautifulSoup
import lxml
import re
import pandas as pd

In [3]:
from IPython.core.interactiveshell import InteractiveShell

# Set interactive shell to display all output
InteractiveShell.ast_node_interactivity = "all"


In [4]:
page = requests.get("https://data.un.org/")
soup = BeautifulSoup(page.content, 'lxml')
main_page = soup.find("div", {"class": "CountryList"})
countries = [item.text.strip().split("\n") for item in main_page if len(item.text.strip()) > 1  ]
countries_links = [item["href"] for item in main_page.find_all("a",href=True)]


In [6]:
all_values_dict = {}
all_countries_data={}
for country,country_link  in enumerate(countries_links):
    page2 = requests.get(f"https://data.un.org/{country_link}")
    soup2 = BeautifulSoup(page2.content, 'lxml')
    main_page2 = soup2.find_all('tr')
    all_values_dict = {}  # Initialize the dictionary to store the values

    for value in main_page2:
        if len(value('td')) == 4:  # To make sure that the lists don't go out of range
            year_2010 = value('td')[1].text.strip()
            year_2015 = value('td')[2].text.strip()
            year_2021 = value('td')[3].text.strip()
            all_values_dict[value('td')[0].text.strip()] = [year_2010, year_2015, year_2021]
        elif len(value('td')) >= 3:  # Ensure there are at least 3 'td' elements for the else condition
            all_values_dict[value('td')[0].text.strip()] = value('td')[2].text.strip()

    all_countries_data[countries[0][country]] = all_values_dict  # Assign the dictionary to the specified key
with open('data.json', 'w') as fp:
    json.dump(all_countries_data, fp)